In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install geopandas

     |████████████████████████████████| 972kB 2.8MB/s 
     |████████████████████████████████| 10.9MB 15.6MB/s 
     |████████████████████████████████| 14.8MB 307kB/s 


In [4]:
!pip install pickle5
import pickle5 as pickle

In [5]:
import pandas as pd
import numpy as np

import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt
import geopandas as gpd

In [6]:
re = pd.read_csv('/content/drive/My Drive/Colab Notebooks/결과_키움센터추가될좌표.csv')

In [15]:
re.columns = ['gid','행정동','거리']

In [30]:
all = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/동3_인프라필요지수.pkl')
all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83 entries, 0 to 82
Data columns (total 36 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   gid          83 non-null     object  
 1   행정동코드        83 non-null     int64   
 2   행정동          83 non-null     object  
 3   위도           83 non-null     float64 
 4   경도           83 non-null     float64 
 5   학원명          83 non-null     object  
 6   학원개수         83 non-null     float64 
 7   학원tfidf      83 non-null     float64 
 8   시군구명         83 non-null     object  
 9   학교명          83 non-null     object  
 10  학교개수         83 non-null     int64   
 11  초등학생_인구      83 non-null     float64 
 12  30대_인구       83 non-null     float64 
 13  40대_인구       83 non-null     float64 
 14  전체_인구        83 non-null     float64 
 15  geometry     83 non-null     geometry
 16  초등학생인구비율     83 non-null     float64 
 17  키움센터         83 non-null     object  
 18  키움센터_개수      83 non-null     flo

In [31]:
df = pd.merge(all,re,how='left',on=['gid'])
df.fillna('x',inplace=True)

In [33]:
add_list = []
for i in range(len(df)):
  if df['거리'][i]=='x':
    add_list.append(0)
  else :
    add_list.append(1)
df['추가키움'] = add_list
df[['거리','추가키움']].head(20)


,거리,추가키움
0,249.499,1
1,353.583,1
2,559.776,1
3,501.114,1
4,250.557,1
5,0,1
6,250.557,1
7,501.113,1
8,791.983,1
9,559.777,1


In [70]:
df['추가키움'] = df['추가키움'].astype(str)

In [36]:
with open('/content/drive/My Drive/수요지수_최종1012 (2).pkl', 'rb') as f:  
  suyo = pickle.load(f)

In [39]:
# 서울시 키움센터 정원 평균값
(sum(suyo['키움센터_상시돌봄정원'])+sum(suyo['키움센터_일시돌봄정원']))/sum(suyo['키움센터_개수'])
# 28명

28.65934065934066

In [104]:
# 서울시 키움센터 총정원
sum(suyo['총정원']) + 20*3

60862.0

In [72]:
a = df[df['행정동_x']=='화곡제1동']
b = df[df['행정동_x']=='독산제1동']
c = df[df['행정동_x']=='광장동']
b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 17 to 43
Data columns (total 39 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   gid          27 non-null     object  
 1   행정동코드        27 non-null     int64   
 2   행정동_x        27 non-null     object  
 3   위도           27 non-null     float64 
 4   경도           27 non-null     float64 
 5   학원명          27 non-null     object  
 6   학원개수         27 non-null     float64 
 7   학원tfidf      27 non-null     float64 
 8   시군구명         27 non-null     object  
 9   학교명          27 non-null     object  
 10  학교개수         27 non-null     int64   
 11  초등학생_인구      27 non-null     float64 
 12  30대_인구       27 non-null     float64 
 13  40대_인구       27 non-null     float64 
 14  전체_인구        27 non-null     float64 
 15  geometry     27 non-null     geometry
 16  초등학생인구비율     27 non-null     float64 
 17  키움센터         27 non-null     object  
 18  키움센터_개수      27 non-null     fl

In [92]:
add_1 = []
for i in range(len(a)):
  if a['추가키움'][i]=='1':
    add_1.append(28)
  else :
    add_1.append(0)
a['추가정원'] = add_1

In [88]:
add_2 = []
for i in b['거리']:
  if i =='x':
    add_2.append(0)
  else:
    add_2.append(28)

b['추가정원'] = add_2
# add_2

In [89]:
add_3 = []
for i in c['거리']:
  if i =='x':
    add_3.append(0)
  else:
    add_3.append(28)

c['추가정원'] = add_3
# add_2

### 화곡 제1동

In [108]:
a['추가키움'] = a['추가키움'].astype(int)
a['개선된총정원'] = a['총정원']+a['추가정원']
a['개선된총시설수'] = a['총시설수']+a['추가키움']
a['개선된돌봄교실이용비율'] = [(x+1)/(sum(suyo['총정원']) + 20*3) for x in a['개선된총정원']]

In [116]:
a['돌봄교실이용비율'] = [x/sum(a['개선된총정원']) for x in a['개선된총정원']]

tf_a = a['개선된총시설수'] + 1
idf_a = (len(a.gid.unique()) / (len(a)+1))
tfidf_a = tf_a*idf_a
a['센터tfidf_new']= tfidf_a
# tfidf_center
a.columns

Index(['gid', '행정동코드', '행정동_x', '위도', '경도', '학원명', '학원개수', '학원tfidf', '시군구명',
       '학교명', '학교개수', '초등학생_인구', '30대_인구', '40대_인구', '전체_인구', 'geometry',
       '초등학생인구비율', '키움센터', '키움센터_개수', '키움센터_상시돌봄정원', '키움센터_일시돌봄정원', '초등돌봄교실',
       '초등돌봄교실_개수', '초등돌봄교실_교실개수', '초등돌봄교실_이용인원', '지역아동센터', '지역아동센터_개수',
       '지역아동센터_정원', '방과후아카데미_개수', '방과후아카데미_정원', '총정원', '총시설수', '돌봄교실이용비율',
       '센터tfidf', '수요지수', 'percentile', '행정동_y', '거리', '추가키움', '추가정원',
       '개선된총정원', '개선된총시설수', '센터tfidf_new', '개선된돌봄교실이용비율'],
      dtype='object')

In [122]:
a['new_인프라지수'] = (a['센터tfidf_new']* a['개선된돌봄교실이용비율']* a['초등학생인구비율']) /a['학원tfidf'] 

In [123]:
a

,gid,행정동코드,행정동_x,위도,경도,학원명,학원개수,학원tfidf,시군구명,학교명,학교개수,초등학생_인구,30대_인구,40대_인구,전체_인구,geometry,초등학생인구비율,키움센터,키움센터_개수,키움센터_상시돌봄정원,키움센터_일시돌봄정원,초등돌봄교실,초등돌봄교실_개수,초등돌봄교실_교실개수,초등돌봄교실_이용인원,지역아동센터,지역아동센터_개수,지역아동센터_정원,방과후아카데미_개수,방과후아카데미_정원,총정원,총시설수,돌봄교실이용비율,센터tfidf,수요지수,percentile,행정동_y,거리,추가키움,추가정원,개선된총정원,개선된총시설수,센터tfidf_new,개선된돌봄교실이용비율,new_인프라지수
0,다사41ab48ba,1150054000,화곡제1동,37.535080,126.836440,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",35.0,34.000000,강서구,"[서울신원초등학교, 서울월정초등학교, 서울신월초등학교]",3,125.0,310.0,359.0,2085.0,POINT (126.83644 37.53508),0.097809,0,0.0,0.0,0.0,"[서울신원초등학교, 서울월정초등학교, 서울신월초등학교]",3,12.0,258.0,"[강서성결행복한홈스쿨, 푸른나래지역아동센터, 밝은내지역아동센터, 사무엘지역아동센터,...",12.0,296.0,0,0,554.0,15.0,0.064423,15.111111,0.002796,10,화곡제1동,249.499,1,28,582.0,16.0,16.055556,0.009579,0.000442
1,다사41ab48bb,1150054000,화곡제1동,37.537333,126.836420,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",41.0,39.666667,강서구,"[서울강신초등학교, 서울월정초등학교, 서울신월초등학교]",3,64.0,353.0,360.0,2006.0,POINT (126.83642 37.53733),0.050078,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교, 서울강신초등학교]",3,12.0,267.0,"[푸른나래지역아동센터, 밝은내지역아동센터, 사무엘지역아동센터, 단비지역아동센터, 다...",10.0,248.0,0,0,515.0,13.0,0.060106,13.222222,0.000998,4,화곡제1동,353.583,1,28,543.0,14.0,14.166667,0.008938,0.000160
2,다사41ab49aa,1150054000,화곡제1동,37.539586,126.836400,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",50.0,48.166667,강서구,"[서울강신초등학교, 서울월정초등학교, 서울신월초등학교]",3,104.0,430.0,403.0,2288.0,POINT (126.83640 37.53959),0.081377,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교, 서울강신초등학교]",3,12.0,267.0,"[푸른나래지역아동센터, 밝은내지역아동센터, 단비지역아동센터, 예람지역아동센터, 꿈나...",8.0,210.0,0,0,477.0,11.0,0.055900,11.333333,0.001060,6,화곡제1동,559.776,1,28,505.0,12.0,12.277778,0.008314,0.000172
3,다사41ba48aa,1150054000,화곡제1동,37.530589,126.839309,"[청목연세학원, 예일아카데미학원, 본원셀파학원, 트리플·영·수사관학원, 강서이룸학원...",33.0,32.111111,강서구,"[서울신원초등학교, 서울신강초등학교, 서울월정초등학교]",3,68.0,559.0,397.0,2535.0,POINT (126.83931 37.53059),0.053208,0,0.0,0.0,0.0,"[서울신원초등학교, 서울월정초등학교, 서울신강초등학교]",3,10.0,212.0,"[강서성결행복한홈스쿨, 베다니꿈나무지역아동센터, 드림지역아동센터, 꿈나무학교, 옹달...",9.0,227.0,0,0,439.0,12.0,0.051694,12.277778,0.001037,5,화곡제1동,501.114,1,28,467.0,13.0,13.222222,0.007690,0.000168
4,다사41ba48ab,1150054000,화곡제1동,37.532842,126.839289,"[청목연세학원, 예일아카데미학원, 본원셀파학원, 트리플·영·수사관학원, 강서이룸학원...",30.0,29.277778,강서구,"[서울신강초등학교, 서울월정초등학교, 서울신월초등학교]",3,64.0,493.0,350.0,2542.0,POINT (126.83929 37.53284),0.050078,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교, 서울신강초등학교]",3,11.0,238.0,"[강서성결행복한홈스쿨, 베다니꿈나무지역아동센터, 다니엘청소년지역아동센터, 드림지역아...",9.0,197.0,0,0,435.0,12.0,0.051251,12.277778,0.001061,7,화곡제1동,250.557,1,28,463.0,13.0,13.222222,0.007624,0.000172
5,다사41ba48ba,1150054000,화곡제1동,37.535096,126.839270,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",39.0,37.777778,강서구,"[서울월정초등학교, 서울신월초등학교]",2,143.0,479.0,482.0,2890.0,POINT (126.83927 37.53510),0.111894,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교]",2,9.0,192.0,"[강서성결행복한홈스쿨, 밝은내지역아동센터, 단비지역아동센터, 드림지역아동센터, 꿈나...",9.0,217.0,0,0,409.0,11.0,0.048373,11.333333,0.001594,10,화곡제1동,0,1,28,437.0,12.0,12.277778,0.007197,0.000262
6,다사41ba48bb,1150054000,화곡제1동,37.537349,126.839250,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",44.0,42.500000,강서구,"[서울화일초등학교, 서울월정초등학교, 서울신월초등학교]",3,85.0,523.0,396.0,2628.0,POINT (126.83925 37.53735),0.066510,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교, 서울화일초등학교]",3,13.0,281.0,"[밝은내지역아동센터, 단비지역아동센터, 드림지역아동센터, 꿈나무학교, 새하늘지역아동...",9.0,207.0,0,0,488.0,12.0,0.057118,12.277778,0.001089,8,화곡제1동,250.557,1,28,516.0,13.0,13.222222,0.008495,0.000176
7,다사41ba49aa,1150054000,화곡제1동,37.539602,126.839230,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 이지아이영어학원, 대사학원, 강서...",55.0,52.888889,강서구,"[서울화일초등학교, 서울화곡초등학교, 서울월정초등학교, 서울신월초등학교]",4,92.0,517.0,419.0,2553.0,POINT (126.83923 37.53960),0.071987,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교, 서울화일초등학교, 서울화곡초등학교]",4,17.0,372.0,"[밝은내지역아동센터, 꿈나무학교, 새하늘지역아동센터, 좋은친구청소년지역아동센터, 좋...",6.0,130.0,0,0,502.0,10.0,0.058667,10.388889,0.000824,2,화곡제1동,501.113,1,28,530.0,11.0,11.333333,0.008725,0.000135
8,다사41bb47bb,1150054000,화곡제1동,37.528352,126.842158,"[청목연세학원, 예일아카데미학원, 종로학원하늘교육목동서부2관학원, 엘리트탑학원, 트...",51.0,49.111111,강서구,"[서울강

In [121]:
b.columns


Index(['gid', '행정동코드', '행정동_x', '위도', '경도', '학원명', '학원개수', '학원tfidf', '시군구명',
       '학교명', '학교개수', '초등학생_인구', '30대_인구', '40대_인구', '전체_인구', 'geometry',
       '초등학생인구비율', '키움센터', '키움센터_개수', '키움센터_상시돌봄정원', '키움센터_일시돌봄정원', '초등돌봄교실',
       '초등돌봄교실_개수', '초등돌봄교실_교실개수', '초등돌봄교실_이용인원', '지역아동센터', '지역아동센터_개수',
       '지역아동센터_정원', '방과후아카데미_개수', '방과후아카데미_정원', '총정원', '총시설수', '돌봄교실이용비율',
       '센터tfidf', '수요지수', 'percentile', '행정동_y', '거리', '추가키움', '추가정원',
       '개선된총정원', '개선된총시설수', '센터tfidf_new', '개선된돌봄교실이용비율'],
      dtype='object')

### 독산1

In [124]:
b['추가키움'] = b['추가키움'].astype(int)
b['개선된총정원'] = b['총정원']+b['추가정원']
b['개선된총시설수'] = b['총시설수']+b['추가키움']
b['개선된돌봄교실이용비율'] = [(x+1)/(sum(suyo['총정원']) + 20*3) for x in b['개선된총정원']]

tf_b = b['개선된총시설수'] + 1
idf_b = (len(b.gid.unique()) / (len(b)+1))
tfidf_b = tf_b*idf_b
b['센터tfidf_new']= tfidf_b
# tfidf_center
b['new_인프라지수'] = (b['센터tfidf_new']* b['개선된돌봄교실이용비율']* b['초등학생인구비율']) /b['학원tfidf'] 


### 광장동

In [125]:
c['추가키움'] = c['추가키움'].astype(int)
c['개선된총정원'] = c['총정원']+c['추가정원']
c['개선된총시설수'] = c['총시설수']+c['추가키움']
c['개선된돌봄교실이용비율'] = [(x+1)/(sum(suyo['총정원']) + 20*3) for x in c['개선된총정원']]

tf_c = c['개선된총시설수'] + 1
idf_c = (len(c.gid.unique()) / (len(c)+1))
tfidf_c = tf_c*idf_c
c['센터tfidf_new']= tfidf_c
# tfidf_center
c['new_인프라지수'] = (c['센터tfidf_new']* c['개선된돌봄교실이용비율']* c['초등학생인구비율']) /c['학원tfidf'] 


In [120]:
a.shape,b.shape,c.shape

((17, 44), (27, 44), (39, 44))

In [129]:
lst = [a,b,c,]
new_index = pd.concat(lst)
len(new_index)

(83, 54)

In [130]:
new_index.to_pickle('동3개_개선인프라지수.pickle')

## 기존 동3 인프라 지수 구하기

In [ ]:
with open('/content/drive/My Drive/동3개_초등학교881m_학원정보.pkl', 'rb') as f:  
  aca = pickle.load(f)

  # /content/drive/My Drive/동3개_초등학교881m_돌봄시설정보.pkl

In [ ]:
with open('/content/drive/My Drive/동3개_초등학교881m_인구.pkl', 'rb') as f:  
  pop = pickle.load(f)

In [ ]:
with open('/content/drive/My Drive/동3개_초등학교881m_돌봄시설정보.pkl', 'rb') as f:  
  dol = pickle.load(f)
 
# /content/drive/My Drive/금천구_초등학교881m_돌봄시설정보.pkl

In [ ]:
aca.info()
aca.fillna(0,inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83 entries, 0 to 82
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   gid     83 non-null     object 
 1   행정동코드   83 non-null     int64  
 2   행정동     83 non-null     object 
 3   위도      83 non-null     float64
 4   경도      83 non-null     float64
 5   학원명     82 non-null     object 
 6   학원개수    82 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 5.2+ KB


In [ ]:
aca

,gid,행정동코드,행정동,위도,경도,학원명,학원개수
0,다사41ab48ba,1150054000,화곡제1동,37.535080,126.836440,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",35.0
1,다사41ab48bb,1150054000,화곡제1동,37.537333,126.836420,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",41.0
2,다사41ab49aa,1150054000,화곡제1동,37.539586,126.836400,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",50.0
3,다사41ba48aa,1150054000,화곡제1동,37.530589,126.839309,"[청목연세학원, 예일아카데미학원, 본원셀파학원, 트리플·영·수사관학원, 강서이룸학원...",33.0
4,다사41ba48ab,1150054000,화곡제1동,37.532842,126.839289,"[청목연세학원, 예일아카데미학원, 본원셀파학원, 트리플·영·수사관학원, 강서이룸학원...",30.0
...,...,...,...,...,...,...,...
78,다사65ab50ab,1121581000,광장동,37.552067,127.107999,"[서울관악보습학원, 더채움영수아카데미학원, 이엠플러스(EM+)영수학원]",3.0
79,다사65ab50ba,1121581000,광장동,37.554320,127.107987,"[서울관악보습학원, 이엠플러스(EM+)영수학원]",2.0
80,다사65ab50bb,1121581000,광장동,37.556574,127.107975,[서울관악보습학원],1.0
81,다사65ba49bb,1121581000,광장동,37.547570,127.110853,"[서울관악보습학원, 더채움영수아카데미학원, 이엠플러스(EM+)영수학원]",3.0


In [ ]:
a = aca[aca['행정동']=='화곡제1동']
b = aca[aca['행정동']=='독산제1동']
c = aca[aca['행정동']=='광장동']


In [ ]:
# aca.행정동.unique()
a

,gid,행정동코드,행정동,위도,경도,학원명,학원개수
0,다사41ab48ba,1150054000,화곡제1동,37.535080,126.836440,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",35.0
1,다사41ab48bb,1150054000,화곡제1동,37.537333,126.836420,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",41.0
2,다사41ab49aa,1150054000,화곡제1동,37.539586,126.836400,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",50.0
3,다사41ba48aa,1150054000,화곡제1동,37.530589,126.839309,"[청목연세학원, 예일아카데미학원, 본원셀파학원, 트리플·영·수사관학원, 강서이룸학원...",33.0
4,다사41ba48ab,1150054000,화곡제1동,37.532842,126.839289,"[청목연세학원, 예일아카데미학원, 본원셀파학원, 트리플·영·수사관학원, 강서이룸학원...",30.0
5,다사41ba48ba,1150054000,화곡제1동,37.535096,126.839270,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",39.0
6,다사41ba48bb,1150054000,화곡제1동,37.537349,126.839250,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",44.0
7,다사41ba49aa,1150054000,화곡제1동,37.539602,126.839230,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 이지아이영어학원, 대사학원, 강서...",55.0
8,다사41bb47bb,1150054000,화곡제1동,37.528352,126.842158,"[청목연세학원, 예일아카데미학원, 종로학원하늘교육목동서부2관학원, 엘리트탑학원, 트...",51.0
9,다사41bb48aa,1150054000,화곡제1동,37.530605,126.842139,"[청목연세학원, 예일아카데미학원, 본원셀파학원, 트리플·영·수사관학원, 강서이룸학원...",39.0


In [ ]:
sum(a['학원개수']),sum(b['학원개수']),sum(c['학원개수'])


(703.0, 455.0, 902.0)

In [ ]:
cnt_a = 0
for i in range(len(a['학원개수'])):
    if aca['학원개수'][i] != 0:
        cnt_a += 1
cnt_a ,len(a)

(17, 17)

In [ ]:
cnt_b = 0
for i in range(len(b['학원개수'])):
    if aca['학원개수'][i] != 0:
        cnt_b += 1
cnt_b,len(b)

(27, 27)

In [ ]:
c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 44 to 82
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   gid      39 non-null     object 
 1   행정동코드    39 non-null     int64  
 2   행정동      39 non-null     object 
 3   위도       39 non-null     float64
 4   경도       39 non-null     float64
 5   학원명      39 non-null     object 
 6   학원개수     39 non-null     float64
 7   학원tfidf  39 non-null     float64
dtypes: float64(4), int64(1), object(3)
memory usage: 2.7+ KB


In [ ]:
# cnt_c = 0
# for i in range(len(c['학원개수'])):
#     if aca['학원개수'][i] != 0:
#         cnt_c += 1
cnt_c = len(c) -1

In [ ]:
tf_aca = [(float(a['학원개수'][i] + 1)) for i in range(len(a))]
idf_aca = ((len(a.gid.unique())) / (cnt_a+1))
tfidf_aca = [x*idf_aca for x in tf_aca]
a['학원tfidf'] = tfidf_aca 
# idf_aca
# tfidf_aca
a

,gid,행정동코드,행정동,위도,경도,학원명,학원개수,학원tfidf
0,다사41ab48ba,1150054000,화곡제1동,37.535080,126.836440,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",35.0,34.000000
1,다사41ab48bb,1150054000,화곡제1동,37.537333,126.836420,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",41.0,39.666667
2,다사41ab49aa,1150054000,화곡제1동,37.539586,126.836400,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",50.0,48.166667
3,다사41ba48aa,1150054000,화곡제1동,37.530589,126.839309,"[청목연세학원, 예일아카데미학원, 본원셀파학원, 트리플·영·수사관학원, 강서이룸학원...",33.0,32.111111
4,다사41ba48ab,1150054000,화곡제1동,37.532842,126.839289,"[청목연세학원, 예일아카데미학원, 본원셀파학원, 트리플·영·수사관학원, 강서이룸학원...",30.0,29.277778
5,다사41ba48ba,1150054000,화곡제1동,37.535096,126.839270,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",39.0,37.777778
6,다사41ba48bb,1150054000,화곡제1동,37.537349,126.839250,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",44.0,42.500000
7,다사41ba49aa,1150054000,화곡제1동,37.539602,126.839230,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 이지아이영어학원, 대사학원, 강서...",55.0,52.888889
8,다사41bb47bb,1150054000,화곡제1동,37.528352,126.842158,"[청목연세학원, 예일아카데미학원, 종로학원하늘교육목동서부2관학원, 엘리트탑학원, 트...",51.0,49.111111
9,다사41bb48aa,1150054000,화곡제1동,37.530605,126.842139,"[청목연세학원, 예일아카데미학원, 본원셀파학원, 트리플·영·수사관학원, 강서이룸학원...",39.0,37.777778


In [ ]:
tf_b = b['학원개수']+1
idf_aca_b = ((len(b.gid.unique())) / (cnt_b+1))
tfidf_aca_b = tf_b*idf_aca_b 
b['학원tfidf'] = tfidf_aca_b 
# tfidf_aca_b
# idf_aca_b
# tf_b
b

,gid,행정동코드,행정동,위도,경도,학원명,학원개수,학원tfidf
17,다사45bb39bb,1154561000,독산제1동,37.456490,126.888015,"[엠베스트에스이(SE)에이스학원, 메가에스디(SD)학원, 뮤엠영어유클리드수학전문학원...",9.0,9.642857
18,다사45bb40aa,1154561000,독산제1동,37.458743,126.887997,"[엠베스트에스이(SE)에이스학원, 메가에스디(SD)학원, (주)지유에듀테인먼트원격학...",10.0,10.607143
19,다사45bb40ab,1154561000,독산제1동,37.460996,126.887979,"[엠베스트에스이(SE)에이스학원, 메가에스디(SD)학원, 네임드2관학원, 네임드학원...",12.0,12.535714
20,다사46aa39ba,1154561000,독산제1동,37.454251,126.890860,"[엠베스트에스이(SE)에이스학원, 메가에스디(SD)학원, 뮤엠영어유클리드수학전문학원...",11.0,11.571429
21,다사46aa39bb,1154561000,독산제1동,37.456504,126.890842,"[엠베스트에스이(SE)에이스학원, 메가에스디(SD)학원, 뮤엠영어유클리드수학전문학원...",11.0,11.571429
22,다사46aa40aa,1154561000,독산제1동,37.458758,126.890824,"[엠베스트에스이(SE)에이스학원, 메가에스디(SD)학원, (주)지유에듀테인먼트원격학...",12.0,12.535714
23,다사46aa40ab,1154561000,독산제1동,37.461011,126.890805,"[엠베스트에스이(SE)에이스학원, 메가에스디(SD)학원, 네임드2관학원, 네임드학원...",16.0,16.392857
24,다사46aa40ba,1154561000,독산제1동,37.463264,126.890787,"[엠베스트에스이(SE)에이스학원, 메가에스디(SD)학원, 네임드2관학원, 네임드학원...",16.0,16.392857
25,다사46aa40bb,1154561000,독산제1동,37.465518,126.890769,"[엠베스트에스이(SE)에이스학원, 네임드2관학원, 네임드학원, 플러스하다학원, 솔로...",14.0,14.464286
26,다사46aa41aa,1154561000,독산제1동,37.467771,126.890751,"[엠베스트에스이(SE)에이스학원, 네임드2관학원, 네임드학원, 아이넷스쿨원격학원, ...",10.0,10.607143


In [ ]:
tf_c = c['학원개수']+1
idf_aca_c = ((len(c.gid.unique())) / (cnt_c+1))
tfidf_aca_c = tf_c*idf_aca_c 
c['학원tfidf'] = tfidf_aca_c 
# tfidf_aca_b
# idf_aca_b
# tf_b
c

,gid,행정동코드,행정동,위도,경도,학원명,학원개수,학원tfidf
44,다사64aa49ab,1121581000,광장동,37.543006,127.093896,"[생명을살리는수학창의교육보습학원, 플랜에이(A)보습학원, 청출어람보습학원, 해강학원...",63.0,64.0
45,다사64ab49aa,1121581000,광장동,37.540762,127.096739,"[생명을살리는수학창의교육보습학원, 청출어람보습학원, 해강학원, 씨원수학학원, 21세...",54.0,55.0
46,다사64ab49ab,1121581000,광장동,37.543016,127.096726,"[생명을살리는수학창의교육보습학원, 청출어람보습학원, 해강학원, 씨원수학학원, 21세...",53.0,54.0
47,다사64ab49ba,1121581000,광장동,37.545269,127.096714,"[생명을살리는수학창의교육보습학원, 청출어람보습학원, 해강학원, 씨원수학학원, 21세...",60.0,61.0
48,다사64ba48bb,1121581000,광장동,37.538519,127.099580,"[생명을살리는수학창의교육보습학원, 청출어람보습학원, 해강학원, 씨원수학학원, 21세...",51.0,52.0
49,다사64ba49aa,1121581000,광장동,37.540772,127.099568,"[생명을살리는수학창의교육보습학원, 청출어람보습학원, 해강학원, 씨원수학학원, 21세...",51.0,52.0
50,다사64ba49ab,1121581000,광장동,37.543025,127.099556,"[생명을살리는수학창의교육보습학원, 청출어람보습학원, 해강학원, 씨원수학학원, 21세...",51.0,52.0
51,다사64ba49ba,1121581000,광장동,37.545279,127.099544,"[생명을살리는수학창의교육보습학원, 청출어람보습학원, 서울관악보습학원, 해강학원, 씨...",55.0,56.0
52,다사64ba49bb,1121581000,광장동,37.547532,127.099532,"[청출어람보습학원, 서울관악보습학원, 해강학원, 씨원수학학원, 21세기보습학원, 와...",50.0,51.0
53,다사64ba50aa,1121581000,광장동,37.549785,127.099520,"[서울관악보습학원, 신연세보습학원, 이황학원, 제이아카데미학원, 매썬스수학과학보습학...",13.0,14.0


In [ ]:
lst = [a,b,c]
academy_all = pd.concat(lst,axis=0)
academy_all

,gid,행정동코드,행정동,위도,경도,학원명,학원개수,학원tfidf
0,다사41ab48ba,1150054000,화곡제1동,37.535080,126.836440,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",35.0,34.000000
1,다사41ab48bb,1150054000,화곡제1동,37.537333,126.836420,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",41.0,39.666667
2,다사41ab49aa,1150054000,화곡제1동,37.539586,126.836400,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",50.0,48.166667
3,다사41ba48aa,1150054000,화곡제1동,37.530589,126.839309,"[청목연세학원, 예일아카데미학원, 본원셀파학원, 트리플·영·수사관학원, 강서이룸학원...",33.0,32.111111
4,다사41ba48ab,1150054000,화곡제1동,37.532842,126.839289,"[청목연세학원, 예일아카데미학원, 본원셀파학원, 트리플·영·수사관학원, 강서이룸학원...",30.0,29.277778
...,...,...,...,...,...,...,...,...
78,다사65ab50ab,1121581000,광장동,37.552067,127.107999,"[서울관악보습학원, 더채움영수아카데미학원, 이엠플러스(EM+)영수학원]",3.0,4.000000
79,다사65ab50ba,1121581000,광장동,37.554320,127.107987,"[서울관악보습학원, 이엠플러스(EM+)영수학원]",2.0,3.000000
80,다사65ab50bb,1121581000,광장동,37.556574,127.107975,[서울관악보습학원],1.0,2.000000
81,다사65ba49bb,1121581000,광장동,37.547570,127.110853,"[서울관악보습학원, 더채움영수아카데미학원, 이엠플러스(EM+)영수학원]",3.0,4.000000


In [ ]:
academy_all.행정동.unique()

array(['화곡제1동', '독산제1동', '광장동'], dtype=object)

In [ ]:
hwa = pop[pop['행정동']=='화곡제1동']
dok = pop[pop['행정동']=='독산제1동']
gwang = pop[pop['행정동']=='광장동']

In [ ]:
sum(hwa['초등학생_인구'])

1278.0

In [ ]:
dol.fillna(0,inplace=True)

In [ ]:
popcnt = 0
for i in range(len(pop['초등학생_인구'])):
    if pop['초등학생_인구'][i] != 0:
        popcnt += pop['초등학생_인구'][i]
popcnt

6178.0

In [ ]:
dol['총정원'] = dol['키움센터_상시돌봄정원']+dol['키움센터_일시돌봄정원']+dol['초등돌봄교실_이용인원']+dol['지역아동센터_정원']+dol['방과후아카데미_정원']
dol['총시설수'] = dol['키움센터_개수']+dol['초등돌봄교실_개수']+dol['지역아동센터_개수']+dol['방과후아카데미_개수']

In [ ]:
dol.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83 entries, 0 to 82
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   gid          83 non-null     object 
 1   행정동코드        83 non-null     int64  
 2   행정동          83 non-null     object 
 3   위도           83 non-null     float64
 4   경도           83 non-null     float64
 5   키움센터         83 non-null     object 
 6   키움센터_개수      83 non-null     float64
 7   키움센터_상시돌봄정원  83 non-null     float64
 8   키움센터_일시돌봄정원  83 non-null     float64
 9   초등돌봄교실       83 non-null     object 
 10  초등돌봄교실_개수    83 non-null     int64  
 11  초등돌봄교실_교실개수  83 non-null     float64
 12  초등돌봄교실_이용인원  83 non-null     float64
 13  지역아동센터       83 non-null     object 
 14  지역아동센터_개수    83 non-null     float64
 15  지역아동센터_정원    83 non-null     float64
 16  방과후아카데미_개수   83 non-null     int64  
 17  방과후아카데미_정원   83 non-null     int64  
 18  총정원          83 non-null     float64
 19  총시설수      

In [ ]:
hwa = dol[dol['행정동']=='화곡제1동']
dok = dol[dol['행정동']=='독산제1동']
gwang = dol[dol['행정동']=='광장동']

In [ ]:
sum(hwa['총정원']),sum(dok['총정원']),sum(gwang['총정원'])

(8614.0, 11402.0, 10819.0)

In [ ]:
hwa['돌봄교실이용비율'] = [x/sum(hwa['총정원']) for x in hwa['총정원']]
dok['돌봄교실이용비율'] = [x/sum(dok['총정원']) for x in dok['총정원']]
gwang['돌봄교실이용비율'] = [x/sum(gwang['총정원']) for x in gwang['총정원']]
tf_hwa = hwa['총시설수'] + 1
idf_center = (len(hwa.gid.unique()) / (hwa_cnt+1))
tfidf_center =tf_hwa*idf_center
hwa['센터tfidf']= tfidf_center
# tfidf_center


In [ ]:
# gwang

In [ ]:
#  tf_center = [float(all['총학원수'][i]+1) / float(all['총학원수'].sum()) for i in range(len(all))]
#  idf = float(len(all.행정동코드.unique()) / (423))
#  tfidf = [x*idf for x in tf]

In [ ]:
hwa_cnt = len(hwa)
dok_cnt = len(dok)
gwang_cnt = len(gwang)

In [ ]:
tf_hwa = hwa['총시설수'] + 1
idf_center = (len(hwa.gid.unique()) / (hwa_cnt+1))
tfidf_center =tf_hwa*idf_center
hwa['센터tfidf']= tfidf_center
# tfidf_center

In [ ]:
tf_dok = dok['총시설수'] + 1
idf_dok = (len(dok.gid.unique()) / (dok_cnt+1))
tfidf_dok =tf_dok*idf_dok
dok['센터tfidf']= tfidf_dok

In [ ]:
tf_gwang = gwang['총시설수'] + 1
idf_gwang = (len(gwang.gid.unique()) / (gwang_cnt+1))
tfidf_gwang =tf_gwang*idf_gwang
gwang['센터tfidf']= tfidf_gwang

In [ ]:
dong = [hwa , dok, gwang]
dol_df = pd.concat(dong)
dol_df

,gid,행정동코드,행정동,위도,경도,키움센터,키움센터_개수,키움센터_상시돌봄정원,키움센터_일시돌봄정원,초등돌봄교실,초등돌봄교실_개수,초등돌봄교실_교실개수,초등돌봄교실_이용인원,지역아동센터,지역아동센터_개수,지역아동센터_정원,방과후아카데미_개수,방과후아카데미_정원,총정원,총시설수,돌봄교실이용비율,센터tfidf
0,다사41ab48ba,1150054000,화곡제1동,37.535080,126.836440,0,0.0,0.0,0.0,"[서울신원초등학교, 서울월정초등학교, 서울신월초등학교]",3,12.0,258.0,"[강서성결행복한홈스쿨, 푸른나래지역아동센터, 밝은내지역아동센터, 사무엘지역아동센터,...",12.0,296.0,0,0,554.0,15.0,0.064314,15.111111
1,다사41ab48bb,1150054000,화곡제1동,37.537333,126.836420,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교, 서울강신초등학교]",3,12.0,267.0,"[푸른나래지역아동센터, 밝은내지역아동센터, 사무엘지역아동센터, 단비지역아동센터, 다...",10.0,248.0,0,0,515.0,13.0,0.059786,13.222222
2,다사41ab49aa,1150054000,화곡제1동,37.539586,126.836400,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교, 서울강신초등학교]",3,12.0,267.0,"[푸른나래지역아동센터, 밝은내지역아동센터, 단비지역아동센터, 예람지역아동센터, 꿈나...",8.0,210.0,0,0,477.0,11.0,0.055375,11.333333
3,다사41ba48aa,1150054000,화곡제1동,37.530589,126.839309,0,0.0,0.0,0.0,"[서울신원초등학교, 서울월정초등학교, 서울신강초등학교]",3,10.0,212.0,"[강서성결행복한홈스쿨, 베다니꿈나무지역아동센터, 드림지역아동센터, 꿈나무학교, 옹달...",9.0,227.0,0,0,439.0,12.0,0.050964,12.277778
4,다사41ba48ab,1150054000,화곡제1동,37.532842,126.839289,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교, 서울신강초등학교]",3,11.0,238.0,"[강서성결행복한홈스쿨, 베다니꿈나무지역아동센터, 다니엘청소년지역아동센터, 드림지역아...",9.0,197.0,0,0,435.0,12.0,0.050499,12.277778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,다사65ab50ab,1121581000,광장동,37.552067,127.107999,0,0.0,0.0,0.0,"[서울광장초등학교, 서울동의초등학교]",2,6.0,144.0,0,0.0,0.0,0,0,144.0,2.0,0.013310,2.925000
79,다사65ab50ba,1121581000,광장동,37.554320,127.107987,0,0.0,0.0,0.0,"[서울광장초등학교, 서울동의초등학교]",2,6.0,144.0,0,0.0,0.0,0,0,144.0,2.0,0.013310,2.925000
80,다사65ab50bb,1121581000,광장동,37.556574,127.107975,0,0.0,0.0,0.0,[서울동의초등학교],1,4.0,97.0,0,0.0,0.0,0,0,97.0,1.0,0.008966,1.950000
81,다사65ba49bb,1121581000,광장동,37.547570,127.110853,0,0.0,0.0,0.0,[서울광장초등학교],1,2.0,47.0,0,0.0,0.0,0,0,47.0,1.0,0.004344,1.950000


In [ ]:
p1 = pop[pop['행정동']=='화곡제1동']
p2 = pop[pop['행정동']=='독산제1동']
p3 = pop[pop['행정동']=='광장동']
p1['초등학생인구비율'] = p1['초등학생_인구']/(sum(p1['초등학생_인구']))
p2['초등학생인구비율'] = p2['초등학생_인구']/(sum(p2['초등학생_인구']))
p3['초등학생인구비율'] = p3['초등학생_인구']/(sum(p3['초등학생_인구']))
pop_lst = [p1,p2,p3]
population = pd.concat(pop_lst)

In [ ]:
len(academy_all),len(dol_df),len(population)

(83, 83, 83)

In [ ]:
df1 = pd.merge(academy_all,population,on=['gid'])
df2 = pd.merge(df1,dol_df,on=['gid'])
df2.columns

Index(['gid', '행정동코드_x', '행정동_x', '위도_x', '경도_x', '학원명', '학원개수', '학원tfidf',
       '행정동코드_y', '시군구명', '행정동_y', '학교명', '학교개수', '초등학생_인구', '30대_인구',
       '40대_인구', '전체_인구', '경도_y', '위도_y', 'geometry', '초등학생인구비율', '행정동코드',
       '행정동', '위도', '경도', '키움센터', '키움센터_개수', '키움센터_상시돌봄정원', '키움센터_일시돌봄정원',
       '초등돌봄교실', '초등돌봄교실_개수', '초등돌봄교실_교실개수', '초등돌봄교실_이용인원', '지역아동센터',
       '지역아동센터_개수', '지역아동센터_정원', '방과후아카데미_개수', '방과후아카데미_정원', '총정원', '총시설수',
       '돌봄교실이용비율', '센터tfidf'],
      dtype='object')

In [ ]:
df2 = df2[['gid', '행정동코드_x', '행정동_x', '위도_x', '경도_x', '학원명', '학원개수', '학원tfidf',
       '시군구명', '학교명', '학교개수', '초등학생_인구', '30대_인구',
       '40대_인구', '전체_인구', 'geometry', '초등학생인구비율','키움센터', '키움센터_개수', '키움센터_상시돌봄정원', '키움센터_일시돌봄정원',
       '초등돌봄교실', '초등돌봄교실_개수', '초등돌봄교실_교실개수', '초등돌봄교실_이용인원', '지역아동센터',
       '지역아동센터_개수', '지역아동센터_정원', '방과후아카데미_개수', '방과후아카데미_정원', '총정원', '총시설수',
       '돌봄교실이용비율', '센터tfidf']]

In [ ]:
df2.columns =['gid', '행정동코드', '행정동', '위도', '경도', '학원명', '학원개수', '학원tfidf',
       '시군구명', '학교명', '학교개수', '초등학생_인구', '30대_인구',
       '40대_인구', '전체_인구', 'geometry', '초등학생인구비율','키움센터', '키움센터_개수', '키움센터_상시돌봄정원', '키움센터_일시돌봄정원',
       '초등돌봄교실', '초등돌봄교실_개수', '초등돌봄교실_교실개수', '초등돌봄교실_이용인원', '지역아동센터',
       '지역아동센터_개수', '지역아동센터_정원', '방과후아카데미_개수', '방과후아카데미_정원', '총정원', '총시설수',
       '돌봄교실이용비율', '센터tfidf']

In [ ]:
# ratio =  [(i+1) / sum(df2['초등학생_인구']) for i in df2['초등학생_인구']]
# df2['인구비율'] = ratio

In [ ]:
df2['수요지수'] = df2['센터tfidf'] * df2['초등학생인구비율']* df2['돌봄교실이용비율'] / df2['학원tfidf']

In [ ]:
df2.행정동.unique()

array(['화곡제1동', '독산제1동', '광장동'], dtype=object)

In [ ]:
d1 = df2[df2['행정동']=='화곡제1동']
d2 = df2[df2['행정동']=='독산제1동']
d3 = df2[df2['행정동']== '광장동']

In [ ]:
pt1 = np.percentile(d1[['수요지수']],[i*10 for i in range(1,10)]).tolist()
pt2 = np.percentile(d2[['수요지수']],[i*10 for i in range(1,10)]).tolist()
pt3 = np.percentile(d3[['수요지수']],[i*10 for i in range(1,10)]).tolist()

len(pt1)
# df2[df2['percentile'] == '10']

9

In [ ]:
d1['percentile'] = 0
d1.loc[(d1['수요지수'] <= pt1[0]),'percentile'] = '1'
d1.loc[(d1['수요지수'] > pt1[0]) & (d1['수요지수'] <= pt1[1]),'percentile'] = '2'
d1.loc[(d1['수요지수'] > pt1[1]) & (d1['수요지수'] <= pt1[2]),'percentile'] = '3'
d1.loc[(d1['수요지수'] > pt1[2]) & (d1['수요지수'] <= pt1[3]),'percentile'] = '4'
d1.loc[(d1['수요지수'] > pt1[3]) & (d1['수요지수'] <= pt1[4]),'percentile'] = '5'
d1.loc[(d1['수요지수'] > pt1[4]) & (d1['수요지수'] <= pt1[5]),'percentile'] = '6'
d1.loc[(d1['수요지수'] > pt1[5]) & (d1['수요지수'] <= pt1[6]),'percentile'] = '7'
d1.loc[(d1['수요지수'] > pt1[6]) & (d1['수요지수'] <= pt1[7]),'percentile'] = '8'
d1.loc[(d1['수요지수'] > pt1[7]) & (d1['수요지수'] <= pt1[8]),'percentile'] = '9'
d1.loc[(d1['수요지수'] > pt1[8]),'percentile'] = '10'
d1.shape

(17, 36)

In [ ]:
d2['percentile'] = 0
d2.loc[(d2['수요지수'] > pt2[0]) & (d2['수요지수'] <= pt2[1]),'percentile'] = '2'
d2.loc[(d2['수요지수'] > pt2[1]) & (d2['수요지수'] <= pt2[2]),'percentile'] = '3'
d2.loc[(d2['수요지수'] > pt2[2]) & (d2['수요지수'] <= pt2[3]),'percentile'] = '4'
d2.loc[(d2['수요지수'] > pt2[3]) & (d2['수요지수'] <= pt2[4]),'percentile'] = '5'
d2.loc[(d2['수요지수'] > pt2[4]) & (d2['수요지수'] <= pt2[5]),'percentile'] = '6'
d2.loc[(d2['수요지수'] > pt2[5]) & (d2['수요지수'] <= pt2[6]),'percentile'] = '7'
d2.loc[(d2['수요지수'] > pt2[6]) & (d2['수요지수'] <= pt2[7]),'percentile'] = '8'
d2.loc[(d2['수요지수'] > pt2[7]) & (d2['수요지수'] <= pt2[8]),'percentile'] = '9'
d2.loc[(d2['수요지수'] > pt2[8]),'percentile'] = '10'
d2.shape

(27, 36)

In [ ]:
d3['percentile'] = 0
d3.loc[(d3['수요지수'] > pt3[0]) & (d3['수요지수'] <= pt3[1]),'percentile'] = '2'
d3.loc[(d3['수요지수'] > pt3[1]) & (d3['수요지수'] <= pt3[2]),'percentile'] = '3'
d3.loc[(d3['수요지수'] > pt3[2]) & (d3['수요지수'] <= pt3[3]),'percentile'] = '4'
d3.loc[(d3['수요지수'] > pt3[3]) & (d3['수요지수'] <= pt3[4]),'percentile'] = '5'
d3.loc[(d3['수요지수'] > pt3[4]) & (d3['수요지수'] <= pt3[5]),'percentile'] = '6'
d3.loc[(d3['수요지수'] > pt3[5]) & (d3['수요지수'] <= pt3[6]),'percentile'] = '7'
d3.loc[(d3['수요지수'] > pt3[6]) & (d3['수요지수'] <= pt3[7]),'percentile'] = '8'
d3.loc[(d3['수요지수'] > pt3[7]) & (d3['수요지수'] <= pt3[8]),'percentile'] = '9'
d3.loc[(d3['수요지수'] > pt3[8]),'percentile'] = '10'
d3.shape

(39, 36)

In [ ]:
len(d2[d2['percentile'] == '10']),len(d2[d2['percentile'] == '9']),len(d2[d2['percentile'] == '8']),len(d2[d2['percentile'] == '7'])

(3, 3, 2, 3)

In [ ]:
len(d1[d1['percentile'] == '10']),len(d1[d1['percentile'] == '9']),len(d1[d1['percentile'] == '8']),len(d1[d1['percentile'] == '7']),len(d1[d1['percentile'] == '9'])

(2, 2, 1, 2, 2)

In [ ]:
d1
# df2['percentile'] = df2['percentile'].astype(int)

,gid,행정동코드,행정동,위도,경도,학원명,학원개수,학원tfidf,시군구명,학교명,학교개수,초등학생_인구,30대_인구,40대_인구,전체_인구,geometry,초등학생인구비율,키움센터,키움센터_개수,키움센터_상시돌봄정원,키움센터_일시돌봄정원,초등돌봄교실,초등돌봄교실_개수,초등돌봄교실_교실개수,초등돌봄교실_이용인원,지역아동센터,지역아동센터_개수,지역아동센터_정원,방과후아카데미_개수,방과후아카데미_정원,총정원,총시설수,돌봄교실이용비율,센터tfidf,수요지수,percentile
0,다사41ab48ba,1150054000,화곡제1동,37.535080,126.836440,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",35.0,34.000000,강서구,"[서울신원초등학교, 서울월정초등학교, 서울신월초등학교]",3,125.0,310.0,359.0,2085.0,POINT (126.83644 37.53508),0.097809,0,0.0,0.0,0.0,"[서울신원초등학교, 서울월정초등학교, 서울신월초등학교]",3,12.0,258.0,"[강서성결행복한홈스쿨, 푸른나래지역아동센터, 밝은내지역아동센터, 사무엘지역아동센터,...",12.0,296.0,0,0,554.0,15.0,0.064314,15.111111,0.002796,10
1,다사41ab48bb,1150054000,화곡제1동,37.537333,126.836420,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",41.0,39.666667,강서구,"[서울강신초등학교, 서울월정초등학교, 서울신월초등학교]",3,64.0,353.0,360.0,2006.0,POINT (126.83642 37.53733),0.050078,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교, 서울강신초등학교]",3,12.0,267.0,"[푸른나래지역아동센터, 밝은내지역아동센터, 사무엘지역아동센터, 단비지역아동센터, 다...",10.0,248.0,0,0,515.0,13.0,0.059786,13.222222,0.000998,4
2,다사41ab49aa,1150054000,화곡제1동,37.539586,126.836400,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",50.0,48.166667,강서구,"[서울강신초등학교, 서울월정초등학교, 서울신월초등학교]",3,104.0,430.0,403.0,2288.0,POINT (126.83640 37.53959),0.081377,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교, 서울강신초등학교]",3,12.0,267.0,"[푸른나래지역아동센터, 밝은내지역아동센터, 단비지역아동센터, 예람지역아동센터, 꿈나...",8.0,210.0,0,0,477.0,11.0,0.055375,11.333333,0.001060,6
3,다사41ba48aa,1150054000,화곡제1동,37.530589,126.839309,"[청목연세학원, 예일아카데미학원, 본원셀파학원, 트리플·영·수사관학원, 강서이룸학원...",33.0,32.111111,강서구,"[서울신원초등학교, 서울신강초등학교, 서울월정초등학교]",3,68.0,559.0,397.0,2535.0,POINT (126.83931 37.53059),0.053208,0,0.0,0.0,0.0,"[서울신원초등학교, 서울월정초등학교, 서울신강초등학교]",3,10.0,212.0,"[강서성결행복한홈스쿨, 베다니꿈나무지역아동센터, 드림지역아동센터, 꿈나무학교, 옹달...",9.0,227.0,0,0,439.0,12.0,0.050964,12.277778,0.001037,5
4,다사41ba48ab,1150054000,화곡제1동,37.532842,126.839289,"[청목연세학원, 예일아카데미학원, 본원셀파학원, 트리플·영·수사관학원, 강서이룸학원...",30.0,29.277778,강서구,"[서울신강초등학교, 서울월정초등학교, 서울신월초등학교]",3,64.0,493.0,350.0,2542.0,POINT (126.83929 37.53284),0.050078,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교, 서울신강초등학교]",3,11.0,238.0,"[강서성결행복한홈스쿨, 베다니꿈나무지역아동센터, 다니엘청소년지역아동센터, 드림지역아...",9.0,197.0,0,0,435.0,12.0,0.050499,12.277778,0.001061,7
5,다사41ba48ba,1150054000,화곡제1동,37.535096,126.839270,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",39.0,37.777778,강서구,"[서울월정초등학교, 서울신월초등학교]",2,143.0,479.0,482.0,2890.0,POINT (126.83927 37.53510),0.111894,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교]",2,9.0,192.0,"[강서성결행복한홈스쿨, 밝은내지역아동센터, 단비지역아동센터, 드림지역아동센터, 꿈나...",9.0,217.0,0,0,409.0,11.0,0.047481,11.333333,0.001594,10
6,다사41ba48bb,1150054000,화곡제1동,37.537349,126.839250,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",44.0,42.500000,강서구,"[서울화일초등학교, 서울월정초등학교, 서울신월초등학교]",3,85.0,523.0,396.0,2628.0,POINT (126.83925 37.53735),0.066510,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교, 서울화일초등학교]",3,13.0,281.0,"[밝은내지역아동센터, 단비지역아동센터, 드림지역아동센터, 꿈나무학교, 새하늘지역아동...",9.0,207.0,0,0,488.0,12.0,0.056652,12.277778,0.001089,8
7,다사41ba49aa,1150054000,화곡제1동,37.539602,126.839230,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 이지아이영어학원, 대사학원, 강서...",55.0,52.888889,강서구,"[서울화일초등학교, 서울화곡초등학교, 서울월정초등학교, 서울신월초등학교]",4,92.0,517.0,419.0,2553.0,POINT (126.83923 37.53960),0.071987,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교, 서울화일초등학교, 서울화곡초등학교]",4,17.0,372.0,"[밝은내지역아동센터, 꿈나무학교, 새하늘지역아동센터, 좋은친구청소년지역아동센터, 좋...",6.0,130.0,0,0,502.0,10.0,0.058277,10.388889,0.000824,2
8,다사41bb47bb,1150054000,화곡제1동,37.528352,126.842158,"[청목연세학원, 예일아카데미학원, 종로학원하늘교육목동서부2관학원, 엘리트탑학원, 트...",51.0,49.111111,강서구,"[서울강서초등학교, 서울신강초등학교, 서울양강초등학교, 서울양동초등학교]",4,63.0,983.0,584.0,3839.0,POINT (126.84216 37.52835),0.049296,0,0.0,0.0,0.0,"[서울신강초등학교, 서울강서초등학교, 서울양강초등학교, 서울양동초등학교]",4,11.0,270.0,"[강서성결행복한홈스쿨, 베다니꿈나무지역아동센터, 드림지역아동센터, 꿈나무학교, 옹달...",12.0,295.0,0,0,565.0,16.0,0.065591,16.055556,0.001057,5
9,다사41bb48aa,1150054000,화곡제1동,37.530605,126.842139,"[청목연세학원, 예일아카데미학원, 본원셀파학원, 트리플·영·수사관학원, 강서이룸학원...",39.0,37.777778,강서구,"[서울신강초등학교, 서울월정초등학교]",2,85.0,1209.0,758.0,4878.0,POINT (126.84214 37.53060),0.066510,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신강초등학교]",2,7.0

In [ ]:
df_lst = [d1,d2,d3]
df_all = pd.concat(df_lst)
len(df_all)

83

In [ ]:
df_all

,gid,행정동코드,행정동,위도,경도,학원명,학원개수,학원tfidf,시군구명,학교명,학교개수,초등학생_인구,30대_인구,40대_인구,전체_인구,geometry,초등학생인구비율,키움센터,키움센터_개수,키움센터_상시돌봄정원,키움센터_일시돌봄정원,초등돌봄교실,초등돌봄교실_개수,초등돌봄교실_교실개수,초등돌봄교실_이용인원,지역아동센터,지역아동센터_개수,지역아동센터_정원,방과후아카데미_개수,방과후아카데미_정원,총정원,총시설수,돌봄교실이용비율,센터tfidf,수요지수,percentile
0,다사41ab48ba,1150054000,화곡제1동,37.535080,126.836440,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",35.0,34.000000,강서구,"[서울신원초등학교, 서울월정초등학교, 서울신월초등학교]",3,125.0,310.0,359.0,2085.0,POINT (126.83644 37.53508),0.097809,0,0.0,0.0,0.0,"[서울신원초등학교, 서울월정초등학교, 서울신월초등학교]",3,12.0,258.0,"[강서성결행복한홈스쿨, 푸른나래지역아동센터, 밝은내지역아동센터, 사무엘지역아동센터,...",12.0,296.0,0,0,554.0,15.0,0.064314,15.111111,0.002796,10
1,다사41ab48bb,1150054000,화곡제1동,37.537333,126.836420,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",41.0,39.666667,강서구,"[서울강신초등학교, 서울월정초등학교, 서울신월초등학교]",3,64.0,353.0,360.0,2006.0,POINT (126.83642 37.53733),0.050078,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교, 서울강신초등학교]",3,12.0,267.0,"[푸른나래지역아동센터, 밝은내지역아동센터, 사무엘지역아동센터, 단비지역아동센터, 다...",10.0,248.0,0,0,515.0,13.0,0.059786,13.222222,0.000998,4
2,다사41ab49aa,1150054000,화곡제1동,37.539586,126.836400,"[청목연세학원, 예일아카데미학원, 동명아인스학원, 본원셀파학원, 대사학원, 강서이룸...",50.0,48.166667,강서구,"[서울강신초등학교, 서울월정초등학교, 서울신월초등학교]",3,104.0,430.0,403.0,2288.0,POINT (126.83640 37.53959),0.081377,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교, 서울강신초등학교]",3,12.0,267.0,"[푸른나래지역아동센터, 밝은내지역아동센터, 단비지역아동센터, 예람지역아동센터, 꿈나...",8.0,210.0,0,0,477.0,11.0,0.055375,11.333333,0.001060,6
3,다사41ba48aa,1150054000,화곡제1동,37.530589,126.839309,"[청목연세학원, 예일아카데미학원, 본원셀파학원, 트리플·영·수사관학원, 강서이룸학원...",33.0,32.111111,강서구,"[서울신원초등학교, 서울신강초등학교, 서울월정초등학교]",3,68.0,559.0,397.0,2535.0,POINT (126.83931 37.53059),0.053208,0,0.0,0.0,0.0,"[서울신원초등학교, 서울월정초등학교, 서울신강초등학교]",3,10.0,212.0,"[강서성결행복한홈스쿨, 베다니꿈나무지역아동센터, 드림지역아동센터, 꿈나무학교, 옹달...",9.0,227.0,0,0,439.0,12.0,0.050964,12.277778,0.001037,5
4,다사41ba48ab,1150054000,화곡제1동,37.532842,126.839289,"[청목연세학원, 예일아카데미학원, 본원셀파학원, 트리플·영·수사관학원, 강서이룸학원...",30.0,29.277778,강서구,"[서울신강초등학교, 서울월정초등학교, 서울신월초등학교]",3,64.0,493.0,350.0,2542.0,POINT (126.83929 37.53284),0.050078,0,0.0,0.0,0.0,"[서울월정초등학교, 서울신월초등학교, 서울신강초등학교]",3,11.0,238.0,"[강서성결행복한홈스쿨, 베다니꿈나무지역아동센터, 다니엘청소년지역아동센터, 드림지역아...",9.0,197.0,0,0,435.0,12.0,0.050499,12.277778,0.001061,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,다사65ab50ab,1121581000,광장동,37.552067,127.107999,"[서울관악보습학원, 더채움영수아카데미학원, 이엠플러스(EM+)영수학원]",3.0,4.000000,광진구,"[서울광장초등학교, 서울동의초등학교]",2,14.0,43.0,43.0,433.0,POINT (127.10800 37.55207),0.004060,0,0.0,0.0,0.0,"[서울광장초등학교, 서울동의초등학교]",2,6.0,144.0,0,0.0,0.0,0,0,144.0,2.0,0.013310,2.925000,0.000040,7
79,다사65ab50ba,1121581000,광장동,37.554320,127.107987,"[서울관악보습학원, 이엠플러스(EM+)영수학원]",2.0,3.000000,광진구,"[서울광장초등학교, 서울동의초등학교]",2,0.0,0.0,0.0,0.0,POINT (127.10799 37.55432),0.000000,0,0.0,0.0,0.0,"[서울광장초등학교, 서울동의초등학교]",2,6.0,144.0,0,0.0,0.0,0,0,144.0,2.0,0.013310,2.925000,0.000000,0
80,다사65ab50bb,1121581000,광장동,37.556574,127.107975,[서울관악보습학원],1.0,2.000000,광진구,[서울동의초등학교],1,0.0,0.0,0.0,0.0,POINT (127.10798 37.55657),0.000000,0,0.0,0.0,0.0,[서울동의초등학교],1,4.0,97.0,0,0.0,0.0,0,0,97.0,1.0,0.008966,1.950000,0.000000,0
81,다사65ba49bb,1121581000,광장동,37.547570,127.110853,"[서울관악보습학원, 더채움영수아카데미학원, 이엠플러스(EM+)영수학원]",3.0,4.000000,광진구,[서울광장초등학교],1,0.0,0.0,6.0,32.0,POINT (127.11085 37.54757),0.000000,0,0.0,0.0,0.0,[서울광장초등학교],1,2.0,47.0,0,0.0,0.0,0,0,47.0,1.0,0.004344,1.950000,0.000000,0


In [ ]:
df_all.to_pickle('동3_인프라필요지수.pkl')